# Model ResNet

https://www.pluralsight.com/guides/introduction-to-resnet

#### TODOS
1. DONE Debugging, does output make sense?
    1. Resize images
    2. preprocessing fixes
    5. replace scaling by proper function
2. try on leaderboard
3. Include Odometry and fuse into heads
    - Speed
    - Location
4. navigation
5. controller
6. Evaluation on Test set, Modularization



## Dependencies

In [2]:
# MODEL STUFF
import torch
import torch.nn as nn
import torch.optim as optim
#import torch.nn.functional as F
import numpy as np
import torchvision
from torchvision import *
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms

# GENERAL STUFF
import time
import copy
import os
import sys
sys.path.insert(1, 'C:\\Users\\morit\\OneDrive\\UNI\\Master\\WS22\\APP-RAS\\Programming\\data_pipeline') # TODO

# DATA ENGINEERING
from data_sampler import WeightedSampler
from dataset import CARLADataset #, CARLADatasetMultiProcessing
from utils import train_test_split, create_metadata_df

## Model

In [3]:
class MyResnet(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        # ResNet Architecture with pretrained weights, also bigger resnets available
        self.net = torchvision.models.resnet18(weights=True)
        num_ftrs = self.net.fc.in_features

        # Top layer of ResNet which you can modify. We choose Identity to use it as Input for all the heads
        self.net.fc = nn.Identity()
        
        # Input Layer fuer cmd, spd
        self.cmd_input = nn.Sequential(
            nn.Linear(7, 128),
            nn.Tanh(), #nn.LeakyReLU() # TODO
            #nn.Dropout(p=0.5, inplace=False)
        )
        
        self.spd_input = nn.Sequential(
            nn.Linear(1, 128),
            nn.Tanh(), #nn.LeakyReLU() # TODO
            #nn.Dropout(p=0.5, inplace=False)
        )
        
        # MLP
        self.mlp = nn.Sequential(
            nn.Linear(num_ftrs+128+128, num_ftrs+128+128),
            nn.Tanh(), #nn.LeakyReLU()
            #nn.Dropout(p=0.5, inplace=False),
            nn.Linear(num_ftrs+128+128, num_ftrs+128+128),
            nn.Tanh()#, #nn.LeakyReLU()
            #nn.Dropout(p=0.5, inplace=False)
        )
        
        # Regression Heads for Throttle, Brake and Steering
        self.thr_head = nn.Sequential(
            nn.Linear(num_ftrs+128+128, 1),
            nn.Sigmoid() # [0,1] Range Output
            
        )
        
        self.brk_head = nn.Sequential(
            nn.Linear(num_ftrs+128+128, 1),
            nn.Sigmoid() # [0,1] Range Output
            
        )
        
        self.str_head = nn.Sequential(
            nn.Linear(num_ftrs+128+128, 1),
            nn.Tanh() # [-1,1] Range Output
            
        )

    # Forward Pass of the Model
    def forward(self, rgb, cmd, spd):
        rgb = self.net(rgb) # BRG
        cmd = self.cmd_input(cmd)
        spd = self.spd_input(spd)
        
        x = torch.cat((rgb, cmd, spd),1)
        x = self.mlp(x)
        
        #x = self.net.fc(x)
        return self.thr_head(x), self.str_head(x), self.brk_head(x) # 3 Outputs since we have 3 Heads


## Data Loaders, Data Sets

In [4]:
#path_ege_data = os.path.join("..", "..", "data", "Dataset Ege")
train_path = "D:\\data\\Train"
test_path = "D:\\data\\Test"

config = {"used_inputs": ["rgb_prep","measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "seq_len": 1
        }

train_meta = create_metadata_df(train_path, config["used_inputs"])
train_dataset = CARLADataset(root_dir=train_path, config=config,df_meta_data=train_meta)

test_meta = create_metadata_df(test_path, config["used_inputs"])
test_dataset = CARLADataset(root_dir=test_path, config=config,df_meta_data=test_meta)

batch_size = 64
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [5]:
len(train_dataloader)

3664

In [6]:
len(test_dataloader)

382

In [7]:
augumentations = torch.nn.ModuleList([
        transforms.GaussianBlur(9),
        transforms.ColorJitter(brightness=1.0, contrast=0.5, saturation=1, hue=0.1),
        transforms.RandomErasing()
    ])

In [8]:
#mean = torch.tensor([79.6657, 81.5673, 105.6161]) BGR
#std = torch.tensor([66.8309, 60.1001, 66.2220])

mean = torch.tensor([105.6161, 81.5673, 79.6657]) # RGB
std = torch.tensor([66.2220, 60.1001, 66.8309])


transform_norm = transforms.Compose([
    transforms.Normalize(mean, std),
    transforms.Resize([224,224])
])

transform_augument = transforms.RandomApply(augumentations)

## Training

In [9]:
# Initialise Model (GPU or CPU)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
net = MyResnet().cuda() if device else net
net

C:\Users\morit\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


MyResnet(
  (net): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_runnin

In [10]:
def to_cuda_if_possible(data):
    return data.to(device) if device else data

In [37]:
def forward_pass(data, augument = True):
    # further preprocessing
    #X_rgb = transform_norm(torch.squeeze(data["rgb_prep"])).float()
    X_rgb = torch.squeeze(data["rgb_prep"])[:,[2,1,0]].float() # BGR --> RGB
    """
    if False:#augument:
        X_rgb = transform_augument(X_rgb)
    """
    labels = data["command"]
    labels = torch.where(labels == -1, torch.tensor(0), labels).to(torch.int64) # Replace by -1 by 0
    # Convert the labels to a one hot encoded tensor
    one_hot = torch.nn.functional.one_hot(labels, num_classes=7)
    X_cmd = torch.squeeze(one_hot).float()
    X_spd = ((data["speed"]-speed_mean)/speed_std).float()
    
    Y_throttle = data["throttle"].float()
    Y_steer = data["steer"].float()
    Y_brake = data["brake"].float()

    # move to GPU
    X_rgb = to_cuda_if_possible(X_rgb)
    X_cmd = to_cuda_if_possible(X_cmd)
    X_spd = to_cuda_if_possible(X_spd)
    
    Y_throttle = to_cuda_if_possible(Y_throttle)
    Y_steer = to_cuda_if_possible(Y_steer)
    Y_brake = to_cuda_if_possible(Y_brake)

    # compute outputs
    optimizer.zero_grad()

    Y_hat = net(X_rgb, X_cmd, X_spd)
    Y_hat_throttle = to_cuda_if_possible(Y_hat[0])
    Y_hat_steer = to_cuda_if_possible(Y_hat[1])
    Y_hat_brake = to_cuda_if_possible(Y_hat[2])

    # get labels from data
    Y_throttle = to_cuda_if_possible(data["throttle"].float())
    Y_steer = to_cuda_if_possible(data["steer"].float())
    Y_brake = to_cuda_if_possible(data["brake"].float())

    # Calculate Loss
    loss_throttle = 0.5*criterion(Y_hat_throttle, Y_throttle)
    loss_steer = 0.45*criterion(Y_hat_steer, Y_steer)
    loss_brake = 0.05*criterion(Y_hat_brake, Y_brake)
    loss = sum([loss_throttle, loss_steer, loss_brake])
    return loss

In [38]:
# Loss and Optimizer
criterion = nn.L1Loss() ##nn.MSELoss() ##  # Easy to interpret #
optimizer = optim.Adam(net.parameters(), lr=0.0001) #,weight_decay=1e-5 #optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

## Model Trainer Prototype

In [39]:
speed_mean = 2.382234##2.250456762830466
speed_std = 1.724884##0.30215840254891313

In [42]:
%%time

n_epochs = 1
print_every = 1
valid_loss_min = 0.0343#np.Inf
val_loss = []
train_loss = []
total_step = len(train_dataloader)

nan_batches = []

run = True

for epoch in range(1, n_epochs+1):
    
    running_loss = 0.0
    print(f'Epoch {epoch}\n')
    
    # Work through batches
    for batch_idx, data in enumerate(train_dataloader): #data: (['idx', 'rgb', 'speed', 'steer', 'throttle', 'brake'])

        loss = forward_pass(data)
        
        # Backprop
        loss.backward()
        optimizer.step()
        
        loss_value = loss.item()
        
        """
        if np.isnan(loss_value):
            print("nan", batch_idx)
            nan_batches.append((batch_idx, data))
         """   
        
        running_loss += loss_value
        if (batch_idx) % print_every == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss_value ))

        """
        et = time.time()
        print(et-at)
        at = time.time()
        """

        
    # Epoch finished, evaluate network and save if network_learned
    train_loss.append(running_loss/total_step)
    print(f'\ntrain-loss: {np.mean(train_loss):.4f},') # TODO SOLVE NAN ISSUES
    batch_loss = 0

    
    # Evaluation on Test set, skipped for now

    with torch.no_grad():
        net.eval()
        
        val_total_step = len(test_dataloader)
        
        for batch_idx, data in enumerate(test_dataloader):
            
            loss = forward_pass(data, False)
            
            loss_value = loss.item()
            
            if (batch_idx) % print_every == 0:
                print ('Validation [{}/{}], Step [{}/{}], Loss: {:.4f}'
                       .format(epoch, n_epochs, batch_idx, val_total_step, loss_value ))
            
            batch_loss += loss_value
        val_loss.append(batch_loss/len(test_dataloader))
        mean_val_loss = np.mean(val_loss)
        
        print(f'validation loss: {mean_val_loss:.4f}, \n') # TODO SOLVE NAN ISSUES

        network_learned = mean_val_loss < valid_loss_min
        if network_learned:
            valid_loss_min = mean_val_loss
            #torch.save(net.state_dict(), 'resnet'+"_E-"+str(epoch)+'.pth')
            torch.save(net.state_dict(), "resnet_E-6.pth")
            print('Improvement-Detected, save-model')

    # Back to training
    net.train()

Epoch 1

Epoch [1/1], Step [0/3664], Loss: 0.2570
Epoch [1/1], Step [1/3664], Loss: 0.2890
Epoch [1/1], Step [2/3664], Loss: 0.2577
Epoch [1/1], Step [3/3664], Loss: 0.2166
Epoch [1/1], Step [4/3664], Loss: 0.2165
Epoch [1/1], Step [5/3664], Loss: 0.2486
Epoch [1/1], Step [6/3664], Loss: 0.2551
Epoch [1/1], Step [7/3664], Loss: 0.2362
Epoch [1/1], Step [8/3664], Loss: 0.2302
Epoch [1/1], Step [9/3664], Loss: 0.2193
Epoch [1/1], Step [10/3664], Loss: 0.2232
Epoch [1/1], Step [11/3664], Loss: 0.2242
Epoch [1/1], Step [12/3664], Loss: 0.2042
Epoch [1/1], Step [13/3664], Loss: 0.2188
Epoch [1/1], Step [14/3664], Loss: 0.2324
Epoch [1/1], Step [15/3664], Loss: 0.2263
Epoch [1/1], Step [16/3664], Loss: 0.2316
Epoch [1/1], Step [17/3664], Loss: 0.2108
Epoch [1/1], Step [18/3664], Loss: 0.2102
Epoch [1/1], Step [19/3664], Loss: 0.2127
Epoch [1/1], Step [20/3664], Loss: 0.2123


KeyboardInterrupt: 

### TEST

In [24]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
iterator = iter(test_dataloader)

In [27]:
data = next(iterator)

In [32]:
rgb = torch.squeeze(data["rgb_prep"])[:,[2,1,0]]
np.shape(rgb)

torch.Size([64, 3, 160, 960])

In [28]:
rgb = data["rgb_prep"]

In [29]:
np.shape(rgb)

torch.Size([64, 1, 3, 160, 960])

In [20]:
rgb = torch.squeeze(rgb)

In [21]:
np.shape(rgb)

torch.Size([64, 3, 160, 960])

### Test predictions

In [14]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
iterator = iter(test_dataloader)
#print(next(iter(test_dataloader)).keys())

In [ ]:
data = next(iterator)
#data

X_rgb = transform_norm(torch.squeeze(data["rgb"])).float().to(device)
labels = data["command"]
labels = torch.where(labels == -1, torch.tensor(0), labels).to(torch.int64) # Replace by -1 by 0
# Convert the labels to a one hot encoded tensor
one_hot = torch.nn.functional.one_hot(labels, num_classes=7).to(device)
X_cmd = torch.squeeze(one_hot).float().to(device)
X_spd = ((data["speed"]-speed_mean)/speed_std).float().to(device)
#print(np.mean(X_spd.cpu().numpy()))

target_ = (data["throttle"], data["steer"], data["brake"])
with torch.no_grad():
    net.eval()
    outputs_ = net(X_rgb, X_cmd, X_spd)
    
# Durchschnittlicher abs. fehler
for i in [0,1,2]:
    print(np.mean(abs(outputs_[i].cpu().numpy()-target_[i].cpu().numpy())))

In [ ]:
print()

In [ ]:
np.round(outputs_[0].cpu().numpy()-target_[0].cpu().numpy(),2)

Bias Variance

In [ ]:
# Variance 

for i in [0,1,2]:
    outputs = (outputs_[i].cpu().numpy())
    #print(outputs)
    mean_outputs = np.mean(outputs_[i].cpu().numpy())
    #print(mean_outputs)
    diff = (outputs-mean_outputs)**2
    #print(diff)
    value = np.mean(diff)
    print(value)

In [ ]:
# Bias
for i in [0,1,2]:
    targets = (target_[i].cpu().numpy())
    #print(outputs)
    mean_outputs = np.mean(outputs_[i].cpu().numpy())
    #print(mean_outputs)
    diff = outputs-mean_outputs
    #print(diff)
    value = np.mean(diff)
    print(value)

In [ ]:
for i in [0,1,2]:
    print(np.mean(abs(target_[i].cpu().numpy())))
    print(np.std(abs(target_[i].cpu().numpy())))


In [ ]:
i =0

In [ ]:
print(np.round(outputs_[i].cpu().numpy(),1))

In [ ]:
print(np.round(target_[i].cpu().numpy(),1))

### IMG Processing

BGR is now standard FOR carla agent and training

In [ ]:
import cv2
idx, batch = next(enumerate(test_dataloader))
print(batch["rgb"].shape)

In [ ]:
img = batch["rgb"][0]#.shape
img = img.numpy().astype(np.uint8).reshape(160,960,3)
print(img.shape)

#img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) # NUR HIER, NICHT IN CARLA AGENT
print(img.shape)
print(type(img))
transform = transforms.Compose([transforms.ToPILImage()])

tensor = transform(img)

#print(type(tensor))

tensor.show()

#torch.tensor(tensor)

In [ ]:
pil_img = img.astype(np.uint8).reshape(160,960,3)
transform = transforms.Compose([transforms.ToPILImage()])
print(pil_img.shape)
pil_img = transform(pil_img)
pil_img.show()

TEST Normalization

In [ ]:
tensor = transform_norm(torch.squeeze(data["rgb"],1))

In [ ]:
tensor = torch.squeeze(transform_norm(data["rgb"])).float()

In [ ]:
for i in range(64):
    print(np.mean(tensor.numpy()[i], axis = (1,2)))

In [ ]:
np.mean(tensor.numpy(), axis = (0,2,3))

### adding Navigation and speed

In [ ]:
test_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
iterator = iter(test_dataloader)



In [ ]:
data = next(iterator)
#data["speed"]
#data["command"]

Command

In [ ]:
import torch

# Assume labels is a 1D tensor with values from 0 to 6

labels = data["command"]
labels = torch.where(labels == -1, torch.tensor(0), labels) # Replace by -1 by 0
labels = labels.to(torch.int64)

# Convert the labels to a one hot encoded tensor
one_hot = torch.nn.functional.one_hot(labels, num_classes=7)
one_hot = torch.squeeze(one_hot)

print(one_hot.shape)

Speed

In [ ]:
# calc mean over trainingsset
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print(len(train_dataloader))
iterator = iter(train_dataloader)

In [ ]:
i = 0
summe = []
for batch_idx, data in enumerate(train_dataloader):
    #print(data)
    if i % 100 == 0:
        print(i)
    summe.append(np.mean(data["speed"].numpy()))
    i += 1
    if i >= 1000:
        break


In [ ]:
#print(summe)

In [ ]:
print(np.mean(summe)) # 2.2078979146598274
print(np.std(summe)) # 0.22455625005948113
speed_mean = np.mean(summe)
speed_std = np.std(summe)

In [ ]:
batch = next(iterator)
#print(np.round(batch["speed"].numpy(),2))

In [ ]:
(batch["speed"]-speed_mean)/speed_std

In [ ]:
np.mean(((batch["speed"]-speed_mean)/speed_std).numpy())

### Class imbalance

In [ ]:
# calc mean over trainingsset
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
print(len(train_dataloader))
iterator = iter(train_dataloader)

In [ ]:
i = 0
steer = []
throttle = []
brake = []
for batch_idx, data in enumerate(train_dataloader):
    #print(data)
    if i % 100 == 0:
        print(i)
    steer.append(np.mean(data["steer"].numpy()))
    throttle.append(np.mean(data["throttle"].numpy()))
    brake.append(np.mean(data["brake"].numpy()))
    i += 1
    if i >= 100:
        break

In [ ]:
print(np.mean(steer))
print(np.mean(throttle))
print(np.mean(brake))

### Vanishing/Exploding Gradients

In [ ]:
"""
for name, param in net.thr_head.named_parameters():
    if param.requires_grad:
        print(name, param.data.cpu().numpy())
"""

In [ ]:
for name, param in net.spd_input.named_parameters():
    if param.requires_grad:
        print(name, np.max(abs(param.data.cpu().numpy())))

## Saving and Loading

Not suited for leaderboard agents

In [ ]:
#torch.save(net, 'rgb_resnet.pth')

In [ ]:
#net = torch.load('rgb_resnet.pth')

suited for leaderboard agents

In [ ]:
#torch.save(net.state_dict(), "pretrained_1E.pth")

In [ ]:
#net = MyResnet()
#net.load_state_dict(torch.load("resnet_E-4.pth"))
#net.cuda()

## Testing Time

In [ ]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)
idx, X = next(enumerate(test_dataloader))
img = transform_norm(X["rgb"])
img.shape

In [ ]:
torch.squeeze(img,1).shape

In [46]:
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
#print(len(test_dataloader))
at = 0
for batch_idx, data in enumerate(test_dataloader):
    # further preprocessing
    #X_rgb = transform_norm(torch.squeeze(data["rgb_prep"])).float()
    X_rgb = torch.squeeze(data["rgb_prep"])[:,[2,1,0]].float() # BGR --> RGB
    """
    if False:#augument:
        X_rgb = transform_augument(X_rgb)
    """
    labels = data["command"]
    labels = torch.where(labels == -1, torch.tensor(0), labels).to(torch.int64) # Replace by -1 by 0
    # Convert the labels to a one hot encoded tensor
    one_hot = torch.nn.functional.one_hot(labels, num_classes=7)
    X_cmd = torch.squeeze(one_hot).float()
    X_spd = ((data["speed"]-speed_mean)/speed_std).float()
    
    Y_throttle = data["throttle"].float()
    Y_steer = data["steer"].float()
    Y_brake = data["brake"].float()

    # move to GPU
    X_rgb = to_cuda_if_possible(X_rgb)
    X_cmd = to_cuda_if_possible(X_cmd)
    X_spd = to_cuda_if_possible(X_spd)
    
    Y_throttle = to_cuda_if_possible(Y_throttle)
    Y_steer = to_cuda_if_possible(Y_steer)
    Y_brake = to_cuda_if_possible(Y_brake)

    
    et = time.time()
    print(et-at)
    at = time.time()
    


1674333664.0873091
1.7781405448913574
1.7159996032714844
1.7738769054412842
1.708998680114746
1.6113920211791992
1.6854381561279297
1.8790006637573242
1.6829984188079834
1.7120006084442139
1.6170008182525635
1.8352971076965332


KeyboardInterrupt: 

#### Training secounds per batch

1.8778209686279297
1.7550039291381836
1.9759962558746338
2.018435001373291

#### Only dataloader and preprocessing secounds per batch
1.235999584197998
1.2680015563964844
1.3483715057373047
1.2585253715515137
1.1267704963684082
1.124000072479248
1.2410008907318115
1.2269997596740723
1.200000286102295
1.267998456954956
1.2166194915771484
1.2077960968017578
1.2405602931976318
1.2019383907318115

In [49]:
from tqdm import tqdm
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
for batch_idx, data in tqdm(enumerate(test_dataloader)):
    pass


110it [02:39,  1.45s/it]


KeyboardInterrupt: 

# asdf

In [ ]:
"""
# LÖSCHEN
root_dir = "D:\\data\\data"
keep_input = ["lidar", "rgb", "measurements"] # "lidar"

def move_unused_sensors_to_new_folder(root_dir, keep_input):
    for (root, dirs, files) in os.walk(root_dir, topdown=True):
        # Current folder contains the files
        if not dirs:
            dir, input_type = os.path.split(root)
            if input_type not in keep_input:
                path_parts = root.split(os.sep)
                idx_data_first = path_parts.index("data")
                path_parts[idx_data_first + 1] += " unused"
                dir_new = os.path.join(*path_parts)
                if not os.path.exists(dir_new):
                    os.makedirs(dir_new)
                shutil.move(root, dir_new)
                
import shutil
move_unused_sensors_to_new_folder(root_dir,keep_input)
"""

In [ ]:
# Saves preped data in same folder structure under rgb_prep
def rgb_to_disk_2(format):
    assert format in [".npy", ".npz", ".pt"]
    fn_save = np.save if format == ".npy" else np.savez_compressed
    # save npy/ npz
    df_meta = dataset.df_meta_data
    for idx in tqdm(range(len(df_meta))):
        path_parts = dataset.df_meta_data["dir"][idx].split(os.sep)
        # path_parts[path_parts.index("data") + 1] += "_prep_npy"
        dir_name_zip = os.path.join(*path_parts, "rgb_prep")
        if not os.path.exists(dir_name_zip):
            os.makedirs(dir_name_zip)
            # shutil.copytree(os.path.join(dataset.df_meta_data["dir"][idx], "measurements"), os.path.join(*path_parts, "measurements"))
        path = os.path.join(df_meta.iloc[idx][0], "rgb", df_meta.iloc[idx][1])
        img_np = dataset.load_data_from_path(path)
        img_torch = torch.Tensor(img_np)
        img_torch_prep = preprocessing["rgb"](img_torch)
        img_np_prep = img_torch_prep.numpy()
        filename_np = os.path.join(dir_name_zip, f"{df_meta.iloc[idx]['rgb'].split('.')[0]}{format}")
        # torch.save(img_torch_prep, filename_torch)
        with open(filename_np, 'wb') as f:
            fn_save(f, img_np_prep)

In [ ]:
train_path = "D:\\data\\Train"

config = {"used_inputs": ["rgb","measurements"], 
        "used_measurements": ["speed", "steer", "throttle", "brake", "command"],
        "seq_len": 1
        }

dataset = CARLADataset(root_dir=train_path, config=config)

In [ ]:
from tqdm import tqdm
rgb_to_disk_2(".npy")

In [ ]:
dataset.df_meta_data.loc[0]["dir"]